In [1]:
import pandas as pd 

In [2]:
data1 = pd.read_csv("/home4/tuannd/address_handler/data/address_new_notation.csv")
data2 = pd.read_csv("/home4/tuannd/address_handler/data/address_old_notation.csv")

In [3]:
data = pd.concat([data1, data2])
data.head()

,path,transcript
0,/home4/tuannd/ASR_team/data_ASR/dcm/63f2f8911f...,[Lâm Đồng](CITY)
1,/home4/tuannd/ASR_team/data_ASR/dcm/63f2f8911f...,[Bến Cát](WARD) [Bình Dương](CITY)
2,/home4/tuannd/ASR_team/data_ASR/dcm/63f2f8911f...,[tỉnh [Hải Phòng](CITY) quận [Hải An] (DIS)] (...
3,/home4/tuannd/ASR_team/data_ASR/dcm/63f2f8911f...,[Tỉnh [Tiền Giang](CITY)](ADD)
4,/home4/tuannd/ASR_team/data_ASR/dcm/63f2f8911f...,à ở [Thuận An](DIS) [Bình Dương](CITY) (ADD)


In [40]:
import re 
def text_process(text):
    text = text.lower()
    text = text.replace('( ', '(')
    text = text.replace(' )', ')')
    text = text.replace('[ ', '[')
    text = text.replace(' ]', ']')

    text = re.sub(r'\[([^\[]*)\]\(num\)', r'\1', text)
    text = re.sub(r'\[([^\[]*)\]\(date\)', r'\1', text)
    text = re.sub(r'\[([^\[]*)\]\(time\)', r'\1', text)
    text = re.sub(r'\[([^\[]*)\]\(lw\)', r'\1', text)
    # print(text)
    text = text.replace('(add)', '')
    # print(text)
    text = re.sub(r'\[([^\[]*)\]\s*\(([^\(]*)\)', r' [\1] ', text)

    # print(text)
    text = re.sub(r'\[([^\(\[]*)\(([^\(]*)\)\]', r' [\1] ', text)
    text = re.sub(r'[,\.\?\}/\!\n\(\)]', ' ', text)
    # text = re.sub(r'\s+', ' ', text)
    # return text
    new_tokens = []
    tokens = text.split()
    # print(tokens)
    for i in range(len(tokens)):
        token = tokens[i]
        if token in ['[', ']']:
            continue
        if token.startswith('['):
            has_close = False
            j = i 
            while j < len(tokens) - 1:
                if ']' in tokens[j+1]:
                    has_close = True
                    break
                if tokens[j+1].startswith('['):
                    break
                j += 1
            if not has_close:
                print(token)
                token = token.replace('[', '')
        new_tokens.append(token)
    return ' '.join(new_tokens)
text_process('Ở [việt theo](lw) của [Sơn La](CITY)')

ở việt theo của [sơn la](city)
['ở', 'việt', 'theo', 'của', '[sơn', 'la]']


'ở việt theo của [sơn la]'

In [ ]:
data['transcript'] = data['transcript'].apply(text_process)
# data.to_csv("/home4/tuannd/address_handler/data/address_preprocessed.csv", index=False)

In [42]:
data = pd.read_csv("/home4/tuannd/address_handler/data/address_preprocessed.csv")
data['transcript'] = data['transcript'].apply(lambda x: x.replace('[', '').replace(']', ''))
data.to_csv("/home4/tuannd/address_handler/data/address_plain.csv", index=False)

In [ ]:
import librosa 
import soundfile as sf
from p_tqdm import p_map
import os 
SAVE_DIR = "/home4/tuannd/vbee-mfa/data/processed/address/wav_processed"

path_result = data['path'].values
# resample to 16kHz
def resample(path):
    audio, _ = librosa.load(path, sr=16000)
    save_path = os.path.join(SAVE_DIR, os.path.basename(path))
    with open(save_path, 'wb') as f:
        sf.write(f, audio, 16000)

p_map(resample, path_result, num_cpus=15)

In [46]:
data = pd.read_csv("/home4/tuannd/address_handler/data/address_preprocessed.csv")
data['is_exist'] = data['path'].apply(lambda x: os.path.exists(x))

data = data[data['is_exist']]
data.to_csv("/home4/tuannd/address_handler/data/address_preprocessed.csv", index=False)
data['transcript'] = data['transcript'].apply(lambda x: x.replace('[', '').replace(']', ''))
data.to_csv("/home4/tuannd/address_handler/data/address_plain.csv", index=False)